In [1]:
import telepot
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton, ReplyKeyboardMarkup, KeyboardButton
from telepot.delegate import (per_chat_id, per_application, call, create_open, pave_event_space)
from pprint import pprint
import sys
import time
import response_manager
import pandas as pd
#%run Result_Word2Vec.ipynb
%run chatbot.ipynb
#%run w2v_ontology.ipynb

[115355   9418  20922  86895]
[('javax.swing.plaf.basic.BasicDirectoryModel-renameFile(java.io.File-java.io.File)', 'Renames a file in the underlying file system.   ', 0.6291374988057696)]
('javax.imageio.ImageReader-isSeekForwardOnly()', 'Returns true if the current input source has been  marked as seek forward only by passing true as the  seek kForward dOnly argument to the  set tInput method.   ', 0.0)


In [2]:
class ChatBox(telepot.DelegatorBot):
    def __init__(self, token, owner_id):
        self._owner_id = owner_id
        self._seen = set()
        self._store = MessageStore()
        
        super(ChatBox, self).__init__(token, [
            pave_event_space()(
                per_chat_id(), create_open, MessageHandler, self._store, timeout=20),
        ])

In [ ]:
OWNER_ID = 102199992
TOKEN = '2067569208:AAHYKIRPZJ_eKtEFAtAmcFOWGlOMyRbGsXI'

bot = ChatBox(TOKEN, OWNER_ID)
MessageLoop(bot).run_as_thread()
print('Listening ...')

while 1:
    time.sleep(3)

Listening ...
RESULTS:  [(65543955, 0.20302259655570362, ['tkinter', 'random'], 'How to select a random Entry on a tkinter canvas?'), (12972021, 0.19897836850939277, ['django', 'sorting'], 'Sort a list in Django')]
TAGSSS:  [['tkinter', 'random'], ['django', 'sorting']]


Traceback (most recent call last):
  File "/Users/marcolilliu/.local/share/virtualenvs/Progetto_NLP-CF9aZTjh/lib/python3.8/site-packages/telepot/delegate.py", line 269, in wait_loop
    j.on_message(msg)
  File "/Users/marcolilliu/.local/share/virtualenvs/Progetto_NLP-CF9aZTjh/lib/python3.8/site-packages/telepot/helper.py", line 689, in augmented
    return handler(msg)
  File "/Users/marcolilliu/.local/share/virtualenvs/Progetto_NLP-CF9aZTjh/lib/python3.8/site-packages/telepot/helper.py", line 1067, in on_message
    self._router.route(msg)
  File "/Users/marcolilliu/.local/share/virtualenvs/Progetto_NLP-CF9aZTjh/lib/python3.8/site-packages/telepot/helper.py", line 1041, in route
    return fn(msg, *args, **kwargs)
  File "/Users/marcolilliu/.local/share/virtualenvs/Progetto_NLP-CF9aZTjh/lib/python3.8/site-packages/telepot/helper.py", line 1049, in <lambda>
    self._router = Router(flavor, {'chat': lambda msg: self.on_chat_message(msg),
  File "/var/folders/7j/b9v_1c594f74fpz8g452mpf

RESULTS:  [117294  39089  39091  39092]
